Table A13: Confidence in work to tackle alcohol and/or drug related issues in Northern Ireland

In [1]:
from databaker.framework import *
import pandas as pd 

In [2]:
import requests
from cachecontrol import CacheControl
from cachecontrol.caches.file_cache import FileCache
from cachecontrol.heuristics import LastModified
from pathlib import Path

session = CacheControl(requests.Session(),
                       cache=FileCache('.cache'),
                       heuristic=LastModified())

sourceFolder = Path('in')
sourceFolder.mkdir(exist_ok=True)

inputURL = 'https://www.justice-ni.gov.uk/sites/default/files/publications/justice/ni-omnibus-survey-oct-2016-alcohol-drugs-tabular.xlsx'
inputFile = sourceFolder / 'ni-omnibus-survey-oct-2016-alcohol-drugs-tabular.xlsx'
Value = session.get(inputURL)
with open(inputFile, 'wb') as f:
  f.write(Value.content)

https://www.justice-ni.gov.uk/sites/default/files/publications/justice/ni-omnibus-survey-oct-2016-alcohol-drugs-tabular.xlsx

In [3]:
tab = loadxlstabs(inputFile, sheetids='Table A13')[0]

Loading in\ni-omnibus-survey-oct-2016-alcohol-drugs-tabular.xlsx which has size 191159 bytes
Table names: ['Table A13']


In [4]:
observations = tab.excel_ref('B5').expand(DOWN).expand(RIGHT).is_not_blank()

In [5]:
observations

{<E12 786.0>, <C10 0.441>, <C9 0.499>, <C12 0.343>, <C11 0.497>, <B10 0.113>, <E9 924.0>, <B11 0.177>, <D5 0.472>, <B9 0.259>, <E5 917.0>, <B5 0.094>, <E10 887.0>, <D8 0.422>, <E8 914.0>, <B12 0.074>, <C5 0.434>, <C8 0.461>, <E11 823.0>, <D9 0.241>, <D10 0.446>, <B8 0.118>, <D11 0.326>, <D12 0.583>}

In [6]:
Confidenceinwork = tab.excel_ref('B3').expand(RIGHT).is_not_blank()
Confidenceinwork

{<D3 'Little or no confidence'>, <B3 'Total or a lot of confidence'>, <C3 'Some confidence'>, <E3 'Unweighted base'>}

In [7]:
publicbody = tab.excel_ref('A5').expand(DOWN).is_not_blank()
publicbody

{<A12 'NIHE / HA'>, <A9 'PSNI'>, <A5 'Overall'>, <A10 'Local Council'>, <A15 'Source: Northern Ireland Omnibus Survey, October 2016'>, <A11 'PCSP'>, <A16 '1. Results exclude "don\'t know" and refusals.'>, <A8 'Gvt Depts'>}

In [8]:
Dimensions = [
            HDim(Confidenceinwork,'Alcohol related issue response',DIRECTLY,ABOVE),
            HDim(publicbody,'Representatives',DIRECTLY,LEFT),
            HDimConst('Measure Type', 'Percent'),
            HDimConst('Unit','People'),
            ]

In [9]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
# savepreviewhtml(c1)

In [10]:
Final_table = c1.topandas()
Final_table

,OBS,Alcohol related issue response,Representatives,Measure Type,Unit
0,0.094,Total or a lot of confidence,Overall,Percent,People
1,0.434,Some confidence,Overall,Percent,People
2,0.472,Little or no confidence,Overall,Percent,People
3,917.000,Unweighted base,Overall,Percent,People
4,0.118,Total or a lot of confidence,Gvt Depts,Percent,People
5,0.461,Some confidence,Gvt Depts,Percent,People
6,0.422,Little or no confidence,Gvt Depts,Percent,People
7,914.000,Unweighted base,Gvt Depts,Percent,People
8,0.259,Total or a lot of confidence,PSNI,Percent,People
9,0.499,Some confidence,PSNI,Percent,People


In [11]:
Final_table.columns = ['Value' if x=='OBS' else x for x in Final_table.columns]

In [12]:
Final_table['Value'] = pd.to_numeric(Final_table['Value'], errors='coerce')

In [13]:
Final_table['Value'] = Final_table['Value'].apply(str)

In [14]:
def user_perc(x,y):
    
    if str(x) == 'Unweighted base':
        return 'Count'
    else:
        return y
    
Final_table['Measure Type'] = Final_table.apply(lambda row: user_perc(row['Alcohol related issue response'],row['Measure Type']), axis = 1)

In [15]:
Final_table['Representatives'].unique()

array(['Overall', 'Gvt Depts', 'PSNI', 'Local Council', 'PCSP',
       'NIHE / HA'], dtype=object)

In [16]:
Final_table['Representatives'] = Final_table['Representatives'].map(
    lambda x: {
        'Overall' : 'overall', 
        'Gvt Depts' : 'govt-dept',
        'PSNI': 'police' ,
        'PCSP': 'pcsp',
        'NIHE / HA': 'nihe',
        'Local Council': 'councillor'        
        }.get(x, x))

In [17]:
Final_table['Alcohol related issue response'].unique()

array(['Total or a lot of confidence', 'Some confidence',
       'Little or no confidence', 'Unweighted base'], dtype=object)

In [18]:
Final_table['Alcohol related issue response'] = Final_table['Alcohol related issue response'].map(
    lambda x: {
        'Total or a lot of confidence' : 'total-confidence', 
        'Some confidence' : 'some-confidence',
        'Little or no confidence': 'no-confidence' ,
        'Unweighted base': 'unweighted-base'
       }.get(x, x))

In [19]:
Final_table.head()

,Value,Alcohol related issue response,Representatives,Measure Type,Unit
0,0.094,total-confidence,overall,Percent,People
1,0.434,some-confidence,overall,Percent,People
2,0.472,no-confidence,overall,Percent,People
3,917.0,unweighted-base,overall,Count,People
4,0.118,total-confidence,govt-dept,Percent,People


In [20]:
Final_table = Final_table[['Representatives','Alcohol related issue response','Measure Type','Value','Unit']]

In [21]:
destinationFolder = Path('out')
destinationFolder.mkdir(exist_ok=True, parents=True)

Final_table.to_csv(destinationFolder / ('table13.csv'), index = False)